In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-11-03"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
13496,2024-11-03,Espanha Acb,08:30,Barcelona,Leyma Coruna,1.07,8.60,171.5,1.84,1.90,-16.5,1.99,1.76,fH6kVHmR,0.934579,0.116279,0.543478,0.526316,0.050859,0.6,0.0,NaN,NaN,146.354,45.325766,0.309700,NaN,NaN,NaN,133.716,0.000,113.03,267.96,0.0,0.0,0.0,0.0,1.101244,0.022688,0.086738,-2.18,-0.436,-0.160550,0.779461,0.5,-0.279461,0.00,0.000,inf,0.000000,0.0,0.000000
13497,2024-11-03,Espanha Acb,13:00,Lleida,MoraBanc Andorra,2.17,1.70,169.5,1.86,1.94,1.5,2.02,1.79,bR4TPjnS,0.460829,0.588235,0.537634,0.515464,0.049065,0.0,0.8,NaN,NaN,NaN,NaN,NaN,201.498,100.433937,0.498436,0.000,321.818,175.96,365.20,0.0,0.0,0.0,0.0,0.171752,0.029773,0.085372,0.00,0.000,inf,0.000000,0.0,0.000000,-5.00,-1.000,-0.700000,0.318301,0.0,-0.318301
13498,2024-11-03,Espanha Acb,14:30,Murcia,Real Madrid,2.68,1.48,165.5,1.80,1.94,3.5,2.04,1.72,QacLPeBl,0.373134,0.675676,0.555556,0.515464,0.048810,0.4,0.6,NaN,NaN,177.340,74.301314,0.418977,117.350,46.765943,0.398517,143.142,140.274,126.72,97.17,0.0,0.0,0.0,0.0,0.407946,0.052938,0.120359,2.18,0.436,3.853211,0.633853,0.7,0.066147,-2.89,-0.578,-0.830450,0.684076,0.4,-0.284076
13499,2024-11-03,Espanha Acb,15:00,Gran Canaria,Baskonia,1.58,2.40,163.5,1.83,1.91,-4.5,1.97,1.78,U9eaKU9k,0.632911,0.416667,0.546448,0.523560,0.049578,0.4,0.8,NaN,NaN,129.166,43.978797,0.340483,198.292,102.846155,0.518660,142.694,216.650,104.80,356.19,0.0,0.0,0.0,0.0,0.291371,0.030251,0.071653,1.95,0.390,1.487179,0.743605,0.8,0.056395,-1.17,-0.234,-5.982906,0.506679,0.3,-0.206679
13500,2024-11-03,Eua Nba,17:30,Brooklyn Nets,Detroit Pistons,1.20,3.81,219.5,1.83,1.83,-9.5,1.96,1.72,UFlkHzk1,0.833333,0.262467,0.546448,0.546448,0.095801,0.8,0.0,NaN,NaN,475.694,182.187803,0.382994,445.190,194.508107,0.436910,314.222,327.700,200.40,257.25,0.0,0.0,0.0,0.0,0.736746,0.000000,0.092231,2.28,0.456,0.438596,0.514392,0.7,0.185608,-0.23,-0.046,-61.086957,0.328548,0.3,-0.028548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13632,2024-11-03,Sérvia Primeira Liga,13:00,KK Mladost SP,OKK Novi Pazar,1.35,2.86,163.5,1.84,1.82,-7.5,1.95,1.72,U34PCZhR,0.740741,0.349650,0.543478,0.549451,0.090391,0.0,0.6,NaN,NaN,NaN,NaN,NaN,176.160,36.509439,0.207252,0.000,297.786,0.00,223.08,0.0,0.0,0.0,0.0,0.507236,0.007728,0.088629,0.00,0.000,inf,0.000000,0.0,0.000000,-0.68,-0.136,-13.676471,0.000000,0.0,0.000000
13633,2024-11-03,Sérvia Primeira Liga,13:30,Cacak 94,Mladost Zemun,1.35,2.86,154.5,1.84,1.82,-7.5,1.95,1.72,Mw1HEeOE,0.740741,0.349650,0.543478,0.549451,0.090391,0.2,0.4,NaN,NaN,135.488,20.932968,0.154501,128.876,34.337607,0.266439,124.706,292.334,117.15,153.00,0.0,0.0,0.0,0.0,0.507236,0.007728,0.088629,-3.74,-0.748,-0.467914,0.559004,0.5,-0.059004,-3.30,-0.660,-2.818182,0.000000,0.0,0.000000
13634,2024-11-03,Sérvia Primeira Liga,13:30,Tamis,Vrsac,2.07,1.64,164.5,1.84,1.82,1.5,1.96,1.71,G0xD3VFD,0.483092,0.609756,0.543478,0.549451,0.092848,0.6,0.4,NaN,NaN,180.006,46.967571,0.260922,147.120,57.189840,0.388729,146.056,178.762,132.24,137.60,0.0,0.0,0.0,0.0,0.163912,0.007728,0.096336,1.64,0.328,3.262195,0.572249,0.5,-0.072249,-0.64,-0.128,-5.000000,0.000000,0.0,0.000000
13635,2024-11-03,Suécia Superettan,10:00,Malbas,Sloga Uppsala,1.69,2.06,158.5,1.85,1.85,-5.5,2.01,1.68,GdG8Mna4,0.591716,0.485437,0.540541,0.540541,0.077153,0.8,0.2,NaN,NaN,11

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,11:00,Rússia Liga Vtb United,MBA Moscow,Parma Perm,150.5,1.8,0.8292,Over
